# Capítulo 10: Cache Locality e TLB

## Introdução

O desempenho de sistemas de banco de dados modernos é frequentemente limitado pela **hierarquia de memória**, não pela CPU. Entender como caches L1/L2/L3 e TLB (Translation Lookaside Buffer) funcionam é essencial para otimizar consultas.

### Objetivos:
1. Entender a hierarquia de cache (L1, L2, L3)
2. Por que o DuckDB usa chunks de ~2048 linhas
3. Medir impacto de cache misses
4. Otimizar acesso à memória para localidade

In [ ]:
!pip install duckdb pandas numpy matplotlib -q

In [ ]:
import duckdb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

plt.rcParams['figure.figsize'] = (12, 6)

## 10.1 Hierarquia de Memória

```
┌─────────────────────────────────────────────────────────────┐
│                     HIERARQUIA DE CACHE                     │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│   Registradores  │  ~1 ciclo   │  ~1 KB    │  Mais rápido  │
│        ↓         │             │           │               │
│   Cache L1       │  ~4 ciclos  │  32-48 KB │               │
│        ↓         │             │           │               │
│   Cache L2       │  ~12 ciclos │  256 KB   │               │
│        ↓         │             │           │               │
│   Cache L3       │  ~40 ciclos │  8-32 MB  │               │
│        ↓         │             │           │               │
│   RAM (DRAM)     │  ~200 ciclos│  GB-TB    │  Mais lento   │
│        ↓         │             │           │               │
│   Disco/SSD      │  ~100K ciclos│  TB      │               │
│                                                             │
└─────────────────────────────────────────────────────────────┘

Latência L1 vs RAM: ~50x diferença!
```

In [ ]:
# Simulação de latências de cache
cache_levels = {
    'L1': {'size_kb': 32, 'latency_cycles': 4, 'latency_ns': 1},
    'L2': {'size_kb': 256, 'latency_cycles': 12, 'latency_ns': 3},
    'L3': {'size_kb': 8192, 'latency_cycles': 40, 'latency_ns': 10},
    'RAM': {'size_kb': 16*1024*1024, 'latency_cycles': 200, 'latency_ns': 50},
}

print("Hierarquia de Cache (valores típicos):")
print(f"{'Nível':<6} {'Tamanho':<12} {'Latência (ciclos)':<20} {'Latência (ns)'}")
print("-" * 55)
for level, info in cache_levels.items():
    size_str = f"{info['size_kb']} KB" if info['size_kb'] < 1024 else f"{info['size_kb']//1024} MB"
    if info['size_kb'] >= 1024*1024:
        size_str = f"{info['size_kb']//1024//1024} GB"
    print(f"{level:<6} {size_str:<12} {info['latency_cycles']:<20} {info['latency_ns']}")

## 10.2 O "Número Mágico" 2048

Por que o DuckDB processa blocos de ~2048 linhas?

```
Cálculo:
  - 3 colunas de int32 (4 bytes) × 2048 linhas = 24 KB
  - Cache L1 típico = 32-48 KB
  - Sobra espaço para vetores auxiliares!

Se usarmos 100.000 linhas:
  - 3 colunas × 100.000 × 4 bytes = 1.2 MB
  - Não cabe nem no L2!
  - Resultado: Cache thrashing, CPU esperando dados
```

In [ ]:
# Demonstração: Por que 2048?

def calculate_chunk_size(num_columns: int, bytes_per_element: int, target_cache_kb: int = 32):
    """Calcula tamanho ideal do chunk para caber no cache L1"""
    target_bytes = target_cache_kb * 1024
    # Deixar 25% de folga para vetores auxiliares
    usable_bytes = target_bytes * 0.75
    bytes_per_row = num_columns * bytes_per_element
    return int(usable_bytes / bytes_per_row)

# Cenários comuns
scenarios = [
    (1, 4, "1 coluna int32"),
    (3, 4, "3 colunas int32"),
    (5, 4, "5 colunas int32"),
    (3, 8, "3 colunas int64/double"),
    (10, 4, "10 colunas int32"),
]

print("Tamanho ideal de chunk para L1 (32 KB):")
print(f"{'Cenário':<25} {'Chunk Size':<12} {'Memória (KB)'}")
print("-" * 50)
for cols, bytes_per, desc in scenarios:
    chunk = calculate_chunk_size(cols, bytes_per)
    mem = chunk * cols * bytes_per / 1024
    print(f"{desc:<25} {chunk:<12} {mem:.1f}")

print(f"\n→ DuckDB usa VECTOR_SIZE = 2048 como padrão")

## 10.3 Benchmark: Impacto do Tamanho do Bloco

In [ ]:
def process_in_chunks(data: np.ndarray, chunk_size: int, operation):
    """Processa dados em chunks do tamanho especificado"""
    n = len(data)
    result = 0
    
    for start in range(0, n, chunk_size):
        end = min(start + chunk_size, n)
        chunk = data[start:end]
        result += operation(chunk)
    
    return result

# Operação de teste: soma com transformação
def complex_operation(arr):
    return np.sum(arr * arr + arr * 2 + 1)

# Benchmark com diferentes tamanhos de chunk
n = 10_000_000
data = np.random.randn(n).astype(np.float64)

chunk_sizes = [64, 128, 256, 512, 1024, 2048, 4096, 8192, 16384, 32768, 65536, 131072]
results = []

for cs in chunk_sizes:
    times = []
    for _ in range(5):
        start = time.perf_counter()
        _ = process_in_chunks(data, cs, complex_operation)
        times.append(time.perf_counter() - start)
    
    results.append({
        'chunk_size': cs,
        'time_ms': np.mean(times) * 1000,
        'memory_kb': cs * 8 / 1024  # float64 = 8 bytes
    })

df = pd.DataFrame(results)
print(df.to_string(index=False))

In [ ]:
# Visualização
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Tempo vs Chunk Size
ax1.plot(df['chunk_size'], df['time_ms'], marker='o', linewidth=2)
ax1.axvline(x=2048, color='red', linestyle='--', label='DuckDB default (2048)')
ax1.axvspan(1024, 4096, alpha=0.2, color='green', label='Zona ótima')
ax1.set_xlabel('Tamanho do Chunk')
ax1.set_ylabel('Tempo (ms)')
ax1.set_title('Tempo de Processamento vs Tamanho do Chunk')
ax1.set_xscale('log', base=2)
ax1.legend()
ax1.grid(True, alpha=0.3)

# Memória do chunk vs níveis de cache
ax2.bar(range(len(chunk_sizes)), df['memory_kb'], color='steelblue')
ax2.axhline(y=32, color='red', linestyle='--', linewidth=2, label='L1 Cache (32 KB)')
ax2.axhline(y=256, color='orange', linestyle='--', linewidth=2, label='L2 Cache (256 KB)')
ax2.set_xticks(range(len(chunk_sizes)))
ax2.set_xticklabels([str(cs) for cs in chunk_sizes], rotation=45)
ax2.set_xlabel('Tamanho do Chunk')
ax2.set_ylabel('Memória do Chunk (KB)')
ax2.set_title('Memória por Chunk vs Níveis de Cache')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 10.4 Localidade Espacial vs Temporal

```
LOCALIDADE ESPACIAL:
  Acessar dados próximos na memória.
  Arrays são ótimos! A[i] e A[i+1] estão lado a lado.
  
  BOM:  for i in range(n): sum += arr[i]     ← Acesso sequencial
  RUIM: for i in range(n): sum += arr[random]  ← Acesso aleatório

LOCALIDADE TEMPORAL:
  Reusar dados recentemente acessados.
  Chunks pequenos permitem reusar dados no cache!
  
  DuckDB: Processa TODO o pipeline em um chunk antes de ir para o próximo.
```

In [ ]:
# Demonstração: Acesso sequencial vs aleatório

n = 10_000_000
data = np.random.randn(n)

# Gerar índices aleatórios
random_indices = np.random.permutation(n)

# Acesso sequencial
times_seq = []
for _ in range(5):
    start = time.perf_counter()
    total = 0
    for i in range(n):
        total += data[i]
    times_seq.append(time.perf_counter() - start)

# Acesso aleatório
times_random = []
for _ in range(5):
    start = time.perf_counter()
    total = 0
    for i in range(n):
        total += data[random_indices[i]]
    times_random.append(time.perf_counter() - start)

# Vetorizado (NumPy) - para comparação
times_vec = []
for _ in range(5):
    start = time.perf_counter()
    total = np.sum(data)
    times_vec.append(time.perf_counter() - start)

print(f"Soma de {n:,} elementos:")
print(f"  Sequencial:     {np.mean(times_seq)*1000:.1f} ms")
print(f"  Aleatório:      {np.mean(times_random)*1000:.1f} ms")
print(f"  Vetorizado:     {np.mean(times_vec)*1000:.2f} ms")
print(f"\n  Penalidade aleatório: {np.mean(times_random)/np.mean(times_seq):.1f}x mais lento")

## 10.5 Cache Lines e Prefetching

```
CACHE LINE:
  A CPU não carrega 1 byte por vez, carrega uma "linha" de 64 bytes.
  
  Exemplo com int32 (4 bytes):
  ┌────────────────────────────────────────────┐
  │ int[0] │ int[1] │ ... │ int[15] │  = 64 bytes = 1 cache line
  └────────────────────────────────────────────┘
  
  Se você acessa int[0], automaticamente tem int[1] até int[15] no cache!

PREFETCHING:
  A CPU tenta prever quais dados você vai precisar.
  Acesso sequencial = fácil de prever = prefetch eficiente
  Acesso aleatório = impossível prever = muitos stalls
```

In [ ]:
# Demonstração: Stride de acesso e cache lines

def sum_with_stride(arr, stride):
    """Soma elementos pulando 'stride' posições"""
    total = 0
    for i in range(0, len(arr), stride):
        total += arr[i]
    return total

n = 1_000_000
data = np.random.randn(n)

strides = [1, 2, 4, 8, 16, 32, 64, 128, 256]
results = []

for stride in strides:
    times = []
    for _ in range(5):
        start = time.perf_counter()
        _ = sum_with_stride(data, stride)
        times.append(time.perf_counter() - start)
    
    # Normalizar pelo número de elementos acessados
    elements = n // stride
    time_per_element = np.mean(times) / elements * 1e9  # ns
    
    results.append({
        'stride': stride,
        'elements': elements,
        'time_ms': np.mean(times) * 1000,
        'ns_per_element': time_per_element
    })

df = pd.DataFrame(results)
print("Impacto do Stride no acesso:")
print(df.to_string(index=False))

In [ ]:
# Visualização
plt.figure(figsize=(10, 5))
plt.plot(df['stride'], df['ns_per_element'], marker='o', linewidth=2)
plt.axvline(x=8, color='red', linestyle='--', label='8 doubles = 64 bytes (cache line)')
plt.xlabel('Stride (elementos)')
plt.ylabel('Tempo por Elemento (ns)')
plt.title('Impacto do Stride no Desempenho\n(Stride > 8 desperdiça cache lines)')
plt.xscale('log', base=2)
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 10.6 TLB (Translation Lookaside Buffer)

```
O QUE É TLB:
  Cache de traduções de endereços virtuais → físicos.
  Sem TLB, cada acesso à memória precisaria de 4+ acessos extras!

TLB MISS:
  Quando acessamos muitas páginas diferentes (tipicamente 4KB cada),
  o TLB não consegue manter todas as traduções.
  
  Solução: Manter dados em poucas páginas = chunks pequenos!

HUGE PAGES:
  Páginas de 2MB em vez de 4KB = menos entradas no TLB necessárias.
  DuckDB pode usar huge pages para estruturas grandes.
```

In [ ]:
# Demonstração conceitual de TLB

PAGE_SIZE = 4096  # 4 KB típico

def calculate_pages_touched(array_size_bytes, access_pattern='sequential'):
    """Calcula quantas páginas de memória são tocadas"""
    if access_pattern == 'sequential':
        return (array_size_bytes + PAGE_SIZE - 1) // PAGE_SIZE
    else:  # random
        # No pior caso, cada acesso pode tocar uma página diferente
        return array_size_bytes // 8  # Assumindo float64

# Comparar chunks pequenos vs grandes
scenarios = [
    (2048, 3, 8, "DuckDB: 2048 rows × 3 cols × 8 bytes"),
    (10000, 3, 8, "Médio: 10K rows × 3 cols × 8 bytes"),
    (100000, 3, 8, "Grande: 100K rows × 3 cols × 8 bytes"),
    (1000000, 3, 8, "Enorme: 1M rows × 3 cols × 8 bytes"),
]

print("Páginas de memória tocadas por cenário:")
print(f"{'Cenário':<40} {'Tamanho (KB)':<15} {'Páginas (4KB)'}")
print("-" * 70)
for rows, cols, bytes_per, desc in scenarios:
    size_bytes = rows * cols * bytes_per
    pages = calculate_pages_touched(size_bytes)
    print(f"{desc:<40} {size_bytes/1024:<15.1f} {pages}")

print(f"\nTLB típico: ~64-1024 entradas")
print(f"→ Chunks de 2048 linhas: poucas páginas, TLB hits!")
print(f"→ Chunks de 1M linhas: muitas páginas, TLB thrashing!")

## 10.7 DuckDB: Configurações de Memória

In [ ]:
con = duckdb.connect(':memory:')

# Ver configurações de memória
print("Configurações de memória do DuckDB:")
settings = con.execute("""
    SELECT name, value, description 
    FROM duckdb_settings() 
    WHERE name LIKE '%memory%' OR name LIKE '%thread%' OR name LIKE '%buffer%'
""").df()
print(settings.to_string())

In [ ]:
# Benchmark: Impacto do paralelismo e memória

# Criar tabela grande
con.execute("""
    CREATE TABLE benchmark AS
    SELECT 
        i AS id,
        random() * 100 AS val1,
        random() * 100 AS val2,
        random() * 100 AS val3
    FROM generate_series(1, 50000000) AS t(i)
""")
print("Tabela criada com 50M linhas")

In [ ]:
# Benchmark de operações
queries = {
    'Full Scan': 'SELECT COUNT(*) FROM benchmark',
    'Agregação Simples': 'SELECT SUM(val1), AVG(val2) FROM benchmark',
    'Agregação Complexa': 'SELECT SUM(val1 * val2 + val3) FROM benchmark',
    'Com Filtro': 'SELECT SUM(val1) FROM benchmark WHERE val2 > 50',
    'Group By': 'SELECT id % 1000 AS grp, SUM(val1) FROM benchmark GROUP BY grp',
}

print("Performance com 50M linhas:\n")
for name, query in queries.items():
    times = []
    for _ in range(3):
        start = time.perf_counter()
        con.execute(query).fetchall()
        times.append(time.perf_counter() - start)
    
    rows_per_sec = 50_000_000 / np.mean(times) / 1_000_000
    print(f"{name:25} {np.mean(times)*1000:8.1f} ms ({rows_per_sec:.1f}M rows/s)")

In [ ]:
# Ver plano de execução com estatísticas
print("\n=== EXPLAIN ANALYZE ===")
plan = con.execute("""
    EXPLAIN ANALYZE
    SELECT SUM(val1 * val2) 
    FROM benchmark 
    WHERE val3 > 50
""").df()
print(plan.to_string())

## 10.8 Resumo do Curso

### Técnicas de Otimização do DuckDB:

| Capítulo | Técnica | Benefício |
|----------|---------|----------|
| 1 | Execução Vetorizada | Elimina overhead de chamadas virtuais |
| 2 | DataChunk | Estrutura cache-friendly de 2048 linhas |
| 3 | Selection Vectors | Zero-copy filtering |
| 4 | SIMD/Branchless | Processamento paralelo em registradores |
| 5 | Avaliação de Expressões | Resultados intermediários no cache |
| 6 | Hash Aggregation | Scatter/Gather vetorizado |
| 7 | Joins Vetorizados | Prefetching esconde latência |
| 8 | German Strings | Comparação rápida via prefixo inline |
| 9 | Scanners Vetorizados | SIMD unpacking de dados compactados |
| 10 | Cache Locality | Chunks dimensionados para L1 |

### Princípios Gerais:

1. **Processar em batches**: 2048 linhas por vez
2. **Evitar branches**: Usar máscaras e operações SIMD
3. **Manter dados no cache**: Chunks pequenos = L1 hits
4. **Acesso sequencial**: Prefetching e cache lines
5. **Evitar alocações**: Reusar buffers, zero-copy

In [ ]:
# Visualização final: Por que vetorização funciona

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Hierarquia de latência
ax = axes[0, 0]
levels = ['Registrador', 'L1 Cache', 'L2 Cache', 'L3 Cache', 'RAM']
latencies = [1, 4, 12, 40, 200]
colors = ['#27ae60', '#2ecc71', '#f1c40f', '#e67e22', '#e74c3c']
ax.barh(levels, latencies, color=colors)
ax.set_xlabel('Latência (ciclos)')
ax.set_title('Latência por Nível de Memória')
ax.set_xscale('log')
for i, v in enumerate(latencies):
    ax.text(v + 5, i, str(v), va='center')

# 2. Throughput por técnica
ax = axes[0, 1]
techniques = ['Volcano\n(linha a linha)', 'Vetorizado\n(sem SIMD)', 'Vetorizado\n(com SIMD)']
throughput = [1, 10, 40]  # Valores relativos
ax.bar(techniques, throughput, color=['#e74c3c', '#f39c12', '#27ae60'])
ax.set_ylabel('Throughput Relativo')
ax.set_title('Ganho de Performance por Técnica')
ax.set_ylim(0, 50)

# 3. Chunk size vs Cache
ax = axes[1, 0]
chunk_sizes = [512, 1024, 2048, 4096, 8192, 16384]
mem_usage = [cs * 3 * 8 / 1024 for cs in chunk_sizes]  # 3 cols × 8 bytes
ax.bar(range(len(chunk_sizes)), mem_usage, color='steelblue')
ax.axhline(y=32, color='red', linestyle='--', linewidth=2, label='L1 (32KB)')
ax.axhline(y=256, color='orange', linestyle='--', linewidth=2, label='L2 (256KB)')
ax.set_xticks(range(len(chunk_sizes)))
ax.set_xticklabels(chunk_sizes)
ax.set_xlabel('Chunk Size')
ax.set_ylabel('Memória (KB)')
ax.set_title('Memória por Chunk (3 colunas float64)')
ax.legend()

# 4. Resumo de benefícios
ax = axes[1, 1]
benefits = ['Cache Hits', 'SIMD', 'Prefetch', 'Zero-Copy', 'Branchless']
impact = [35, 30, 15, 12, 8]  # Contribuição relativa %
ax.pie(impact, labels=benefits, autopct='%1.0f%%', startangle=90,
       colors=['#3498db', '#2ecc71', '#f1c40f', '#9b59b6', '#e74c3c'])
ax.set_title('Contribuição para Performance')

plt.tight_layout()
plt.savefig('duckdb_optimization_summary.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ Curso completo! Agora você entende como o DuckDB alcança alta performance.")

In [ ]:
con.close()